# CsvHelper Demo

## Initialization and preparation

First, install the `CsvHelper` NuGet package:

In [ ]:
#r "nuget:CsvHelper"

Installed Packages CsvHelper, 28.0.1

Then create some sample data:

In [ ]:
public class ShopItem {
    public string Name { get; set; }
    public string Description { get; set; }
    public decimal Price { get; set; }
    public DateTime DateCreated { get; set; }
}

var items = new[] {
    new ShopItem { 
        Name = "První položka", 
        Description = "Popis.", 
        Price = 123.45M,
        DateCreated = DateTime.Now
    },
    new ShopItem { 
        Name = "Druhá položka", 
        Description = "Popis položky, může obsahovat třeba i \"text v uvozovkách\".", 
        Price = 123.45M,
        DateCreated = DateTime.Now
    },
    new ShopItem { 
        Name = "Třetí položka", 
        Description = "Popis může mít\r\nvíc řádků.", 
        Price = 678.90M,
        DateCreated = DateTime.Now
    },
    new ShopItem { 
        Name = "Čtvrtá položka", 
        Description = null, 
        Price = 543.21M,
        DateCreated = DateTime.Now
    }
};

items

index,Name,Description,Price,DateCreated
0,První položka,Popis.,123.45,2022-08-01 16:03:33Z
1,Druhá položka,"Popis položky, může obsahovat třeba i ""text v uvozovkách"".",123.45,2022-08-01 16:03:33Z
2,Třetí položka,Popis může mít víc řádků.,678.90,2022-08-01 16:03:33Z
3,Čtvrtá položka,<null>,543.21,2022-08-01 16:03:33Z


## Export to CSV

In [ ]:
using System.Globalization;
using System.IO;
using CsvHelper;

using (var writer = new StreamWriter("csvhelper-demo-defaults.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.GetCultureInfo("cs-CZ"))) {
    csv.WriteRecords(items);
}

**See result in [`csvhelper-demo-defaults.csv`](csvhelper-demo-defaults.csv):**

In [ ]:
File.ReadAllText("csvhelper-demo-defaults.csv")

Name;Description;Price;DateCreated
První položka;Popis.;123,45;01.08.2022 16:03:33
Druhá položka;"Popis položky, může obsahovat třeba i ""text v uvozovkách"".";123,45;01.08.2022 16:03:33
Třetí položka;"Popis může mít
víc řádků.";678,90;01.08.2022 16:03:33
Čtvrtá položka;;543,21;01.08.2022 16:03:33


### Change encoding to Windows-1250

It uses default encoding (`UTF-8`) and regionally appropriate delimiters (`;`) and decimal separators (`,`). This is a problem, because Excel by default expects `Windows-1250` encoding.

Current .NET does not support those legacy encodings anymore by default, so we have to install the `System.Text.Encoding.CodePages` package and register these encodings:

In [ ]:
#r "nuget:System.Text.Encoding.CodePages"

using System.Text;
Encoding.RegisterProvider(CodePagesEncodingProvider.Instance);

Installed Packages System.Text.Encoding.CodePages, 6.0.0

Then we can use `Windows-1250` as encoding:

In [ ]:
using CsvHelper.Configuration;

var config = new CsvConfiguration(CultureInfo.GetCultureInfo("cs-CZ")) {
    Encoding = Encoding.GetEncoding("Windows-1250") 
};

using (var writer = new StreamWriter("csvhelper-demo-1250.csv", false, config.Encoding))
using (var csv = new CsvWriter(writer, config)) {
    csv.WriteRecords(items);
}

**Now we have the [`csvhelper-demo-1250.csv`](csvhelper-demo-1250.csv) file**, which is fully compatible with Microsoft Excel in Czech environment. 

> However, it will be not correctly displayed here, as we expect `UTF-8` by default.

In [ ]:
File.ReadAllText("csvhelper-demo-1250.csv").Display();

Name;Description;Price;DateCreated
Prvn� polo�ka;Popis.;123,45;01.08.2022 16:03:33
Druh� polo�ka;"Popis polo�ky, m��e obsahovat t�eba i ""text v uvozovk�ch"".";123,45;01.08.2022 16:03:33
T�et� polo�ka;"Popis m��e m�t
v�c ��dk�.";678,90;01.08.2022 16:03:33
�tvrt� polo�ka;;543,21;01.08.2022 16:03:33


### Configure export options

We can use the `CsvConfiguration` class to configure various aspects of data export. The most common one is setting the culture to invariant one, which changes decimal separators and field separators:

In [ ]:
using (var writer = new StreamWriter("csvhelper-demo-invariant.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture)) {
    csv.WriteRecords(items);
}

**The resulting file is [`csvhelper-demo-invariant.csv`](csvhelper-demo-invariant.csv).** Note the changes below:

In [ ]:
File.ReadAllText("csvhelper-demo-invariant.csv").Display();

Name,Description,Price,DateCreated
První položka,Popis.,123.45,08/01/2022 16:03:33
Druhá položka,"Popis položky, může obsahovat třeba i ""text v uvozovkách"".",123.45,08/01/2022 16:03:33
Třetí položka,"Popis může mít
víc řádků.",678.90,08/01/2022 16:03:33
Čtvrtá položka,,543.21,08/01/2022 16:03:33


Of course, we can change all aspects of the export, like delimiters etc.:

In [ ]:
using CsvHelper.Configuration;

var config = new CsvConfiguration(CultureInfo.GetCultureInfo("cs-CZ")) { 
    HasHeaderRecord = false,    // disable first row with headers
    Delimiter = "|"             // use custom delimiters
};

using (var writer = new StreamWriter("csvhelper-demo-custom.csv"))
using (var csv = new CsvWriter(writer, config)) {
    csv.WriteRecords(items);
}

**See the result in [`csvhelper-demo-custom.csv`](csvhelper-demo-custom.csv):**

In [ ]:
File.ReadAllText("csvhelper-demo-custom.csv").Display();

První položka|Popis.|123,45|01.08.2022 16:03:33
Druhá položka|"Popis položky, může obsahovat třeba i ""text v uvozovkách""."|123,45|01.08.2022 16:03:33
Třetí položka|"Popis může mít
víc řádků."|678,90|01.08.2022 16:03:33
Čtvrtá položka||543,21|01.08.2022 16:03:33


### Use attributes to set column order and names

Lets redefine the `ShopItem` class to `ShopItem2`. We'll add the `Index` attributes to set the column order - move the description to the end. For a good measure, we'll also use `Name` to specify CSV column names.

> By default, fields would be exported in the same order they are defined in the C# code. But it's not always reliable, so if the order is important, specify the index.

In [ ]:
using CsvHelper.Configuration.Attributes;

public class ShopItem2 {

    [Index(0), Name("item_name")]
    public string Name { get; set; }
    
    [Index(3), Name("description")]
    public string Description { get; set; }
    
    [Index(1), Name("price")]
    public decimal Price { get; set; }
    
    [Index(2), Name("date_created")]
    public DateTime DateCreated { get; set; }

}

var items2 = new[] {
    new ShopItem2 { 
        Name = "První položka", 
        Description = "Popis.", 
        Price = 123.45M,
        DateCreated = DateTime.Now
    },
    new ShopItem2 { 
        Name = "Druhá položka", 
        Description = "Popis položky, může obsahovat třeba i \"text v uvozovkách\".", 
        Price = 123.45M,
        DateCreated = DateTime.Now
    },
    new ShopItem2 { 
        Name = "Třetí položka", 
        Description = "Popis může mít\r\nvíc řádků.", 
        Price = 678.90M,
        DateCreated = DateTime.Now
    },
    new ShopItem2 { 
        Name = "Čtvrtá položka", 
        Description = null, 
        Price = 543.21M,
        DateCreated = DateTime.Now
    }
};

When we write them, the column names and order is honored:

In [ ]:
using (var writer = new StreamWriter("csvhelper-demo-attributes.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.GetCultureInfo("cs-CZ"))) {
    csv.WriteRecords(items2);
}

**See resulting file [`csvhelper-demo-attributes.csv`](csvhelper-demo-attributes.csv):**

In [ ]:
File.ReadAllText("csvhelper-demo-attributes.csv").Display();

item_name;price;date_created;description
První položka;123,45;01.08.2022 16:03:33;Popis.
Druhá položka;123,45;01.08.2022 16:03:33;"Popis položky, může obsahovat třeba i ""text v uvozovkách""."
Třetí položka;678,90;01.08.2022 16:03:33;"Popis může mít
víc řádků."
Čtvrtá položka;543,21;01.08.2022 16:03:33;


## Import from CSV

When importing data from CSV, we have to setup mapping between CSV columns and class properties. We can use the `[Index]` and `[Name]` attributes from above, but the most universal way is to create a mapping class. 

Let's have the following class, we would like to populate with data from the last CSV file [`csvhelper-demo-attributes.csv`](csvhelper-demo-attributes.csv):

In [ ]:
public class ImportedItem {
    public string Name { get; set; }
    public string Description { get; set; }
    public decimal Price { get; set; }
    public DateOnly DateCreated { get; set; }
    public TimeOnly TimeCreated { get; set; }
}

There are some problems we need to fix:

* The property `Name` does not match to `item_name` field exactly.
* The property `Description` is in different order than the corresponding field.
* The `date_created` field was split into separate `DateCreated` and `TimeCreated` properties using the new `DateOnly` and `TimeOnly` types. _It does not make much sense, but it's good enough for our demonstration._

We will now write a mapping class, describing how the fields are mapped to properties:

In [ ]:
public class ImportedItemMap : ClassMap<ImportedItem> {

    public ImportedItemMap() {
        // Map using names, that's simple
        this.Map(m => m.Name).Name("item_name");
        this.Map(m => m.Description).Name("description");
        this.Map(m => m.Price).Name("price");

        // Map using some more sophisticated algorithm
        this.Map(m => m.DateCreated).Convert(x => DateOnly.FromDateTime(x.Row.GetField<DateTime>("date_created")));
        this.Map(m => m.TimeCreated).Convert(x => TimeOnly.FromDateTime(x.Row.GetField<DateTime>("date_created")));
    }

}

Then we can use this class to read the file in quite similar fashion we used to create it:

In [ ]:
IList<ImportedItem> importedItems;

using (var reader = new StreamReader("csvhelper-demo-attributes.csv"))
using (var csv = new CsvReader(reader, CultureInfo.GetCultureInfo("cs-CZ"))) {
    csv.Context.RegisterClassMap<ImportedItemMap>();
    importedItems = csv.GetRecords<ImportedItem>().ToList();
}

This is the result we got:

> Do not mind the complex display of `DateOnly` and `TimeOnly` types.

In [ ]:
importedItems.Display();

index Name Description Price DateCreated TimeCreated 0 První položka Popis. 123.45 Year Month Day DayOfWeek DayOfYear DayNumber 2022 8 1 Monday 213 738367 Hour Minute Second Millisecond Ticks 16 3 33 0 578130000000 1 Druhá položka Popis položky, může obsahovat třeba i "text v uvozovkách". 123.45 Year Month Day DayOfWeek DayOfYear DayNumber 2022 8 1 Monday 213 738367 Hour Minute Second Millisecond Ticks 16 3 33 0 578130000000 2 Třetí položka Popis může mít
víc řádků. 678.90 Year Month Day DayOfWeek DayOfYear DayNumber 2022 8 1 Monday 213 738367 Hour Minute Second Millisecond Ticks 16 3 33 0 578130000000 3 Čtvrtá položka 543.21 Year Month Day DayOfWeek DayOfYear DayNumber 2022 8 1 Monday 213 738367 Hour Minute Second Millisecond Ticks 16 3 33 0 578130000000

## Additional resources

This is just a small portion of what CsvHelper can do. For further information see:

* [CsvHelper site](https://joshclose.github.io/CsvHelper/)
* [Getting Started](https://joshclose.github.io/CsvHelper/getting-started/)
* [Examples](https://joshclose.github.io/CsvHelper/examples/)

Sadly, there is no official [API reference documentation](https://joshclose.github.io/CsvHelper/api/) (yet?), but the above resources are enough for usual cases.

## Cleanup

The following cell will delete all files created by this notebook.

In [ ]:
Console.WriteLine("Deleting files:");
var di = new DirectoryInfo(".");
foreach (var fi in di.GetFiles("csvhelper-demo-*.csv")) {
    fi.Delete();
    Console.WriteLine(fi.Name);
}

Deleting files:
csvhelper-demo-1250.csv
csvhelper-demo-attributes.csv
csvhelper-demo-custom.csv
csvhelper-demo-defaults.csv
csvhelper-demo-invariant.csv
